In [80]:
import sys
sys.path.append('src/')

In [81]:
import numpy as np
import torch, torch.nn
from library_function import library_1D_new
from neural_net import LinNetwork
from DeepMod import DeepMod
import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')
import torch.nn as nn
from torch.autograd import grad

# Preparing data

In [82]:
np.random.seed(34) 
number_of_samples = 1000

data = np.load('data/burgers.npy', allow_pickle=True).item()

X = np.transpose((data['x'].flatten(), data['t'].flatten()))
y = np.real(data['u']).reshape((data['u'].size, 1))

idx = np.random.permutation(y.size)
X_train = torch.tensor(X[idx, :][:number_of_samples], dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y[idx, :][:number_of_samples], dtype=torch.float32)

# Building network

In [83]:
network = LinNetwork(input_dim=2, hidden_dim=20, layers=5, output_dim=1)

In [88]:
weight_vector = torch.ones(6, 1, dtype=torch.float32, requires_grad=True)

In [89]:
optimizer = torch.optim.Adam([{'params':network.parameters()}, {'params': weight_vector}])

In [90]:
optimizer.zero_grad()    
prediction = network(X_train)
y = prediction

In [97]:
l1=10**(-5)
loss_L1_pre =  l1*loss_L1(weight_vector,torch.zeros_like(weight_vector))
for iteration in np.arange(10000):
    optimizer.zero_grad()    
    prediction = network(X_train)
    y = prediction
    y_t, theta = library_1D_new(X_train, y,library_config={'poly_order':1,'diff_order':2})
    f = y_t - theta @ weight_vector
    loss_PI = torch.nn.MSELoss()
    loss_MSE = torch.nn.MSELoss()
    loss_L1 = nn.L1Loss()
    loss = loss_MSE(prediction, y_train) + loss_PI(f, torch.zeros_like(f)) + l1*loss_L1(weight_vector,torch.zeros_like(weight_vector))
    
    loss.backward()
    optimizer.step()
    
    if iteration % 100 == 0:
        print(iteration, loss, loss_L1_pre -  loss_L1(weight_vector,torch.zeros_like(weight_vector)))
        loss_L1_pre =  loss_L1(weight_vector,torch.zeros_like(weight_vector))

0 tensor(3.7311e-06, grad_fn=<AddBackward0>) tensor(-0.1818, grad_fn=<SubBackward0>)
100 tensor(3.6724e-06, grad_fn=<AddBackward0>) tensor(-7.3358e-05, grad_fn=<SubBackward0>)
200 tensor(3.6312e-06, grad_fn=<AddBackward0>) tensor(-4.9606e-05, grad_fn=<SubBackward0>)
300 tensor(3.8892e-06, grad_fn=<AddBackward0>) tensor(9.5919e-05, grad_fn=<SubBackward0>)
400 tensor(3.5963e-06, grad_fn=<AddBackward0>) tensor(-0.0001, grad_fn=<SubBackward0>)
500 tensor(5.3377e-06, grad_fn=<AddBackward0>) tensor(-4.4078e-05, grad_fn=<SubBackward0>)
600 tensor(5.0081e-06, grad_fn=<AddBackward0>) tensor(7.3165e-05, grad_fn=<SubBackward0>)
700 tensor(3.8246e-06, grad_fn=<AddBackward0>) tensor(-0.0001, grad_fn=<SubBackward0>)
800 tensor(3.4980e-06, grad_fn=<AddBackward0>) tensor(1.9535e-05, grad_fn=<SubBackward0>)
900 tensor(3.4562e-06, grad_fn=<AddBackward0>) tensor(-5.7116e-05, grad_fn=<SubBackward0>)
1000 tensor(3.4937e-06, grad_fn=<AddBackward0>) tensor(6.1989e-05, grad_fn=<SubBackward0>)
1100 tensor(3.42

In [98]:
scaled_time = torch.norm(y_t).detach().numpy()
scaled_theta = torch.norm(theta,dim=0).detach().numpy()

In [99]:
scaled_weight_vector = np.squeeze(weight_vector.detach().numpy())*scaled_theta

In [101]:
weight_vector

tensor([[ 4.5746e-05],
        [-2.1027e-03],
        [ 9.9532e-02],
        [-5.5832e-04],
        [-9.9276e-01],
        [-1.1922e-03]], requires_grad=True)

In [4]:
DeepMod(X_train, y_train, config={'input_dim':2, 'hidden_dim':10, 'layers':5, 'output_dim':1})

Sequential(
  (0): Linear(in_features=2, out_features=10, bias=True)
  (1): Tanh()
  (2): Linear(in_features=10, out_features=10, bias=True)
  (3): Tanh()
  (4): Linear(in_features=10, out_features=10, bias=True)
  (5): Tanh()
  (6): Linear(in_features=10, out_features=10, bias=True)
  (7): Tanh()
  (8): Linear(in_features=10, out_features=10, bias=True)
  (9): Tanh()
  (10): Linear(in_features=10, out_features=1, bias=True)
)


In [4]:
X = np.arange(0,1000,1).reshape(-1,1)
y = X*X.reshape(-1,1)
idx = np.random.permutation(y.size)

In [5]:
X_train = torch.tensor(X[:, :][:number_of_samples], dtype=torch.float32, requires_grad=True)
y_train = torch.tensor(y[:, :][:number_of_samples], dtype=torch.float32)

In [6]:
network = LinNetwork(input_dim=1, hidden_dim=10, layers=5, output_dim=1)

In [7]:
prediction = network(X_train)
y = prediction

In [8]:
library_1D_new(X_train, y,library_config={'poly_order':2,'diff_order':2})

tensor([[1.0000, 0.1584, 0.0251],
        [1.0000, 0.1471, 0.0217],
        [1.0000, 0.1337, 0.0179],
        [1.0000, 0.1316, 0.0173],
        [1.0000, 0.1339, 0.0179],
        [1.0000, 0.1372, 0.0188],
        [1.0000, 0.1406, 0.0198],
        [1.0000, 0.1437, 0.0207],
        [1.0000, 0.1464, 0.0214],
        [1.0000, 0.1487, 0.0221]], grad_fn=<CatBackward>)
tensor([[ 1.0000e+00, -3.4018e-03, -1.2140e-02],
        [ 1.0000e+00, -1.7684e-02, -2.4011e-03],
        [ 1.0000e+00, -6.8308e-03,  1.3013e-02],
        [ 1.0000e+00,  1.0594e-03,  3.8364e-03],
        [ 1.0000e+00,  3.0818e-03,  8.6494e-04],
        [ 1.0000e+00,  3.4536e-03,  2.6085e-05],
        [ 1.0000e+00,  3.2918e-03, -3.0473e-04],
        [ 1.0000e+00,  2.9105e-03, -4.3198e-04],
        [ 1.0000e+00,  2.4694e-03, -4.3464e-04],
        [ 1.0000e+00,  2.0625e-03, -3.7298e-04]], grad_fn=<CatBackward>)


(tensor([], size=(10, 0), grad_fn=<SliceBackward>),
 tensor([[ 1.0000e+00, -3.4018e-03, -1.2140e-02,  1.0000e+00, -3.4018e-03,
          -1.2140e-02,  1.5843e-01, -5.3895e-04, -1.9233e-03],
         [ 1.0000e+00, -1.7684e-02, -2.4011e-03,  1.0000e+00, -1.7684e-02,
          -2.4011e-03,  1.4715e-01, -2.6021e-03, -3.5332e-04],
         [ 1.0000e+00, -6.8308e-03,  1.3013e-02,  1.0000e+00, -6.8308e-03,
           1.3013e-02,  1.3367e-01, -9.1310e-04,  1.7395e-03],
         [ 1.0000e+00,  1.0594e-03,  3.8364e-03,  1.0000e+00,  1.0594e-03,
           3.8364e-03,  1.3157e-01,  1.3938e-04,  5.0476e-04],
         [ 1.0000e+00,  3.0818e-03,  8.6494e-04,  1.0000e+00,  3.0818e-03,
           8.6494e-04,  1.3388e-01,  4.1260e-04,  1.1580e-04],
         [ 1.0000e+00,  3.4536e-03,  2.6085e-05,  1.0000e+00,  3.4536e-03,
           2.6085e-05,  1.3722e-01,  4.7390e-04,  3.5793e-06],
         [ 1.0000e+00,  3.2918e-03, -3.0473e-04,  1.0000e+00,  3.2918e-03,
          -3.0473e-04,  1.4062e-01,  4.6289e-